<a href="https://colab.research.google.com/github/colaprograms/2019-hackathon-ocr-wymbah/blob/master/notebooks/ctc_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os, torch, re, sys
import numpy as np
import matplotlib.pyplot as p
import random, PIL

In [73]:
os.chdir("/content")
if not os.path.exists("/content/AI4Good---Meza-OCR-Challenge"):
  !git clone https://github.com/Charitable-Analytics-International/AI4Good---Meza-OCR-Challenge
if not os.path.exists("/content/2019-hackathon-ocr-wymbah"):
  !git clone https://github.com/colaprograms/2019-hackathon-ocr-wymbah
os.chdir("/content/2019-hackathon-ocr-wymbah")
!git pull

from importlib import reload
import util.file
reload(util.file)

from google.colab import drive
drive.mount("/content/gdrive")
try:
  os.mkdir("/content/gdrive/My Drive/code")
except:
  pass
try:
  os.mkdir("/content/gdrive/My Drive/code/checkpoint")
except:
  pass

remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 18 (delta 7), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (18/18), done.
From https://github.com/colaprograms/2019-hackathon-ocr-wymbah
   680e840..3199be8  master     -> origin/master
Updating 680e840..3199be8
Fast-forward
 notebooks/ctc_test.ipynb | 204 ++++++++++++-----------------------------------
 util/file.py             |   2 +-
 2 files changed, 51 insertions(+), 155 deletions(-)
Using path /content/AI4Good---Meza-OCR-Challenge
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from util.file import FileHolder
fh = FileHolder()
inp, outp = fh.get_batch(1)

for file, val in zip(inp, outp):
  print(val)
  p.imshow(file)
  p.show()

def to_tensor(buf):
  buf -= 0.5
  buf *= 2
  #buf -= np.mean(buf, axis=(0, 1, 2))[None, None, None, :]
  #buf /= np.std(buf, axis=(0, 1, 2))[None, None, None, :]
  #buf -= np.array([0.485, 0.456, 0.406])[None, None, None, :]
  #buf /= np.array([0.229, 0.224, 0.225])[None, None, None, :]
  return torch.tensor(buf, dtype=torch.float32).permute(0, 3, 1, 2)

def get_batch(batchsize, validation=False):
  inp, outp = fh.get_batch(batchsize, validation)
  inp = np.stack(inp)
  inp = to_tensor(inp)
  return inp, outp

b = get_batch(1)[0].numpy().squeeze().transpose(1, 2, 0)
#print(np.min(b), np.max(b), np.mean(b), np.std(b))
p.imshow(get_batch(1)[0].numpy().squeeze().transpose(1, 2, 0) * 0.5 + 0.5)
p.show()

In [0]:
import torch.nn as nn
import torch.optim as optim
import torchvision



In [170]:
chars = " 0123456789-,."
nchars = len(chars)
idx = {}
for i, c in enumerate(chars): idx[c] = i

ntraining = len(fh.info['training'])

BATCH_SIZE = 64

def maketarget(outputs):
  classes = []
  for string in outputs:
    classes.extend([idx[c] for c in string])
    lengths = [len(string) for string in outputs]
  return torch.tensor(classes, dtype=torch.long), lengths

trainloss = []
validloss = []
class CTCModel(nn.Module):
  def __init__(self):
    super(CTCModel, self).__init__()
    self.conv = nn.Sequential(
      nn.Conv2d(3, 64, (3, 3), padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, (3, 3), stride=2, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, (3, 3), padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 128, (3, 3), stride=2, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Conv2d(128, 128, (3, 1), padding=(1, 0)),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Conv2d(128, 128, (3, 1), stride=2, padding=(1, 0)),
      nn.BatchNorm2d(128),
      nn.ReLU(),
    )
    self.lstm1 = nn.LSTM(128, 256, batch_first=True, bidirectional=True)
    self.layernorm1 = nn.LayerNorm((512,))
    self.dense1 = nn.Sequential(
        nn.Linear(512, 256, bias=False),
        nn.BatchNorm(256),
        nn.ReLU()
    )
    self.dense2 = nn.Sequential(
        nn.Linear(256, 256, bias=False),
        nn.BatchNorm(256),
        nn.ReLU()
    )
    self.dense3 = nn.Sequential(
        nn.Linear(256, nchars, bias=False),
        nn.BatchNorm(nchars),
        nn.ReLU()
    )
    self.avgpool = nn.AdaptiveAvgPool2d((1, 32)).cuda()
  
  def forward(self, z):
    z = self.conv(z)
    z = self.avgpool(z)
    z = z.squeeze(2).permute(0, 2, 1)
    # batch, seq, channels
    z, _ = self.lstm1(z)
    z = self.layernorm1(z)
    z = self.dense1(z)
    z = self.dense2(z)
    z = self.dense3(z)
    print(z[:, 0, 0])
    z = nn.functional.log_softmax(z, dim=2)
    return z
def run():
  """resnet = torchvision.models.resnet.resnet34(True)

  "Cut off the last two layers"
  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    #x = self.layer4(x)

    return x

  import types
  resnet.forward = types.MethodType(forward, resnet)
  resnet = resnet.cuda()
  for param in resnet.parameters():
    param.requires_grad = False
  """
  ctcmodel = CTCModel().cuda()
  ctc = nn.CTCLoss(reduction='sum').cuda()
  optimizer = optim.SGD(ctcmodel.parameters(), lr=0.001, momentum=0.2, nesterov=True, weight_decay=.001)

  def input_to_string(inp):
    assert inp.shape[0] == 1
    inp = inp.detach().cpu().numpy()
    def randchoice(p):
      p = np.exp(p)
      u = random.random()
      for i in range(p.shape[0]):
        u -= p[i]
        if u < 1e-6:
          return i
      raise Exception("not a probability distribution")
    return "".join(chars[randchoice(inp[0, j, :])] for j in range(inp.shape[1]))
  def crunch(inp, outp):
    inp = ctcmodel(inp.cuda())
    inp = inp.permute(1, 0, 2)
    assert inp.shape[0] == 32
    input_lengths = [32 for i in range(inp.shape[1])]
    target, target_lengths = maketarget(outp)
    input_lengths, target_lengths = map(tuple, [input_lengths, target_lengths])
    cu = target.data.cuda()
    #print(inp)
    loss = ctc(inp, cu, input_lengths, target_lengths)
    return loss
  for epoch in range(999999):
    running_loss = 0.0
    length = 0
    print("Example output:")
    inp, outp = get_batch(1)
    with torch.no_grad():
      string = input_to_string(ctcmodel(inp.cuda()))
    print(string, outp)
    for i in range(160):
      inp, outp = get_batch(BATCH_SIZE)
      optimizer.zero_grad()
      loss = crunch(inp, outp)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      length += BATCH_SIZE
    loss = None
    trainingloss = running_loss / length
    print("Epoch %d. Loss: %f" % (epoch, trainingloss))
    trainloss.append(trainingloss)
    running_loss = 0.0
    length = 0
    with torch.no_grad():
      for i in range(20):
        inp, outp = get_batch(BATCH_SIZE, validation=True)
        loss = crunch(inp, outp)
        running_loss += loss.item()
        length += BATCH_SIZE
    validationloss = running_loss / length
    print("Validation loss: %f" % validationloss)
    validloss.append(validationloss)
    file = "/content/gdrive/My Drive/code/checkpoint/checkpoint-%04d-%.2f" % (epoch, validationloss)
    torch.save({
        'ctcmodel': ctcmodel.state_dict(),
        'trainloss': trainloss,
        'validloss': validloss
    }, file)
    print("Saved", file)

run()

Example output:
tensor([0.0365], device='cuda:0')
.5489 8256-680--53-8407288028.65 ['95']
tensor([0.0506, 0.0502, 0.0539, 0.0501, 0.0501, 0.0503, 0.0502, 0.0502, 0.0502,
        0.0549, 0.0548, 0.0501, 0.0510, 0.0502, 0.0501, 0.0566, 0.0627, 0.0504,
        0.0503, 0.0558, 0.0466, 0.0501, 0.0510, 0.0499, 0.0502, 0.0501, 0.0504,
        0.0515, 0.0600, 0.0502, 0.0501, 0.0502, 0.0332, 0.0503, 0.0501, 0.0502,
        0.0502, 0.0501, 0.0504, 0.0527, 0.0502, 0.0502, 0.0507, 0.0560, 0.0511,
        0.0347, 0.0501, 0.0499, 0.0224, 0.0519, 0.0534, 0.0502, 0.0552, 0.0502,
        0.0502, 0.0369, 0.0502, 0.0504, 0.0501, 0.0511, 0.0502, 0.0502, 0.0502,
        0.0502], device='cuda:0', grad_fn=<SelectBackward>)
tensor([37.0511, 37.0924, 37.0725, 37.0977, 37.0862, 37.0933, 37.0897, 37.0927,
        37.0934, 37.0972, 37.0857, 37.0905, 37.0894, 37.0930, 37.0783, 37.0727,
        37.0285, 37.5295, 37.0863, 37.1146, 37.0854, 37.0933, 37.0935, 37.0895,
        37.0931, 37.0884, 37.0896, 37.0438, 37.087

KeyboardInterrupt: ignored

In [0]:
image_tensor = torch.Tensor(image)
image_tensor = torch.unsqueeze(image_tensor, 0)
image_tensor = image_tensor.permute(0, 3, 1, 2)
out = resnet(image_tensor)
print(out)
print(out.shape)

In [0]:
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
'
7878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878787878[pppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppppmjnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx]'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''